In [309]:
#Data cleaning and preprocessing
winedf = pd.read_csv("winequality-red.csv", sep=';')
wineData = winedf.to_numpy()
wineData[:,-1] = (wineData[:,-1]>=6).astype(int) #convert 6+ to 1 and <5 to 0
wineData = wineData/wineData.max(axis=0) #normalize data

bcdf = pd.read_csv("breast-cancer-wisconsin.data", sep=',', header=None)
bcdf.columns = ['Sample Code Number', 'Clump Thickness', 'Uniformity of Cell Size', 'Uniformity of Cell Shape',
                'Marginal Adhesion', 'Single Epithelial Cell Size', 'Bare Nuclei', 'Bland Chromatin',
                'Normal Nucleoli', 'Mitoses', 'Class']
bcdf.replace('?', np.NaN, inplace=True)
bcdf.dropna(inplace=True)
bcdf.drop(['Sample Code Number'], axis=1, inplace=True)
bcData = bcdf.to_numpy().astype(float)
bcData[:,-1] = (bcData[:,-1]>3).astype(int) #change 2/4 last column to 0/1 labels
bcData = bcData/bcData.max(axis=0) #normalize data

In [310]:
winedf

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
5,7.4,0.660,0.00,1.8,0.075,13.0,40.0,0.99780,3.51,0.56,9.4,5
6,7.9,0.600,0.06,1.6,0.069,15.0,59.0,0.99640,3.30,0.46,9.4,5
7,7.3,0.650,0.00,1.2,0.065,15.0,21.0,0.99460,3.39,0.47,10.0,7
8,7.8,0.580,0.02,2.0,0.073,9.0,18.0,0.99680,3.36,0.57,9.5,7
9,7.5,0.500,0.36,6.1,0.071,17.0,102.0,0.99780,3.35,0.80,10.5,5


In [311]:
winedf.sample(frac=1)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
792,7.1,0.610,0.02,2.50,0.081,17.0,87.0,0.99745,3.48,0.60,9.7,6
1346,6.1,0.590,0.01,2.10,0.056,5.0,13.0,0.99472,3.52,0.56,11.4,5
1525,6.7,0.480,0.08,2.10,0.064,18.0,34.0,0.99552,3.33,0.64,9.7,5
602,7.7,0.835,0.00,2.60,0.081,6.0,14.0,0.99750,3.30,0.52,9.3,5
1024,7.7,0.580,0.01,1.80,0.088,12.0,18.0,0.99568,3.32,0.56,10.5,7
807,8.2,0.280,0.40,2.40,0.052,4.0,10.0,0.99356,3.33,0.70,12.8,7
628,8.7,0.540,0.26,2.50,0.097,7.0,31.0,0.99760,3.27,0.60,9.3,6
945,10.2,0.440,0.42,2.00,0.071,7.0,20.0,0.99566,3.14,0.79,11.1,7
1465,6.8,0.590,0.10,1.70,0.063,34.0,53.0,0.99580,3.41,0.67,9.7,5
1085,6.8,0.480,0.08,1.80,0.074,40.0,64.0,0.99529,3.12,0.49,9.6,5


In [312]:
winedf.apply(np.random.permutation)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,11.6,0.320,0.10,2.20,0.063,12.0,26.0,0.99680,3.44,0.66,9.20,5
1,7.7,0.460,0.43,1.60,0.078,17.0,51.0,0.99580,3.19,0.46,12.30,5
2,7.3,0.740,0.50,2.90,0.081,18.0,37.0,0.99786,3.24,0.68,10.20,8
3,9.8,0.490,0.08,6.30,0.070,12.0,89.0,0.99720,3.35,0.46,9.70,6
4,7.1,0.460,0.49,2.10,0.067,5.0,39.0,0.99554,3.26,0.58,11.40,6
5,7.7,0.420,0.09,6.10,0.060,30.0,24.0,0.99580,3.70,0.54,10.80,5
6,10.3,0.520,0.44,2.10,0.076,11.0,16.0,0.99760,3.09,0.66,11.10,6
7,7.5,0.440,0.37,2.10,0.097,4.0,32.0,1.00010,3.15,0.90,11.80,6
8,7.1,0.630,0.49,2.30,0.122,5.0,39.0,0.99680,3.32,0.52,9.50,6
9,9.4,0.330,0.32,2.00,0.073,29.0,11.0,0.99647,3.16,0.59,9.50,5


In [313]:
wineData

array([[0.46540881, 0.44303797, 0.        , ..., 0.28      , 0.63087248,
        0.        ],
       [0.49056604, 0.55696203, 0.        , ..., 0.34      , 0.65771812,
        0.        ],
       [0.49056604, 0.48101266, 0.04      , ..., 0.325     , 0.65771812,
        0.        ],
       ...,
       [0.39622642, 0.32278481, 0.13      , ..., 0.375     , 0.73825503,
        1.        ],
       [0.37106918, 0.40822785, 0.12      , ..., 0.355     , 0.68456376,
        0.        ],
       [0.37735849, 0.19620253, 0.47      , ..., 0.33      , 0.73825503,
        1.        ]])

In [315]:
np.random.shuffle(wineData)

In [318]:
wineData

array([[0.48427673, 0.14556962, 0.37      , ..., 0.355     , 0.81208054,
        1.        ],
       [0.44654088, 0.41772152, 0.        , ..., 0.27      , 0.63758389,
        0.        ],
       [0.52201258, 0.31012658, 0.36      , ..., 0.3       , 0.63758389,
        1.        ],
       ...,
       [0.4591195 , 0.21518987, 0.33      , ..., 0.385     , 0.81208054,
        1.        ],
       [0.40251572, 0.36075949, 0.02      , ..., 0.34      , 0.63758389,
        0.        ],
       [0.63522013, 0.59177215, 0.22      , ..., 0.32      , 0.75838926,
        0.        ]])

In [354]:
a = np.random.random((16,2))
a

array([[0.13197048, 0.68582093],
       [0.63276899, 0.78579744],
       [0.4141059 , 0.22643447],
       [0.72479661, 0.95539378],
       [0.06406458, 0.52054522],
       [0.05448336, 0.37520599],
       [0.34230404, 0.35761793],
       [0.33852436, 0.64159963],
       [0.02506208, 0.00859959],
       [0.15411788, 0.68878295],
       [0.02885691, 0.63069336],
       [0.17730013, 0.32705496],
       [0.38952074, 0.88524601],
       [0.04167986, 0.08812843],
       [0.92373875, 0.52946077],
       [0.99724383, 0.12632496]])

In [391]:
foldsList = np.array_split(a, 4)

In [392]:
for foldIndex, currentFold in enumerate(foldsList):
    print(foldIndex)

0
1
2
3


In [393]:
len(b)

4

In [394]:
a = [1, 2, 4]
b = a[:]
b.insert(2, 3)

In [395]:
a

[1, 2, 4]

In [396]:
b

[1, 2, 3, 4]

In [397]:
a = ['a', 'b']

In [398]:
a

['a', 'b']

In [399]:
a.insert(9, 'c')

In [400]:
a

['a', 'b', 'c']

In [402]:
for foldIndex, currentFold in enumerate(foldsList):
    validationData = currentFold #assign current fold value to validationData
    foldsList.remove(currentFold) #remove the current fold from the list
    trainingData = np.vstack(foldsList)
    # vertically stack the remaining elements in the list creating a matrix of
    # the dataset with validation data removed --> creating the training set
    foldsList.insert(foldIndex, currentFold)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [387]:
validationData

array([[0.06406458, 0.52054522],
       [0.05448336, 0.37520599],
       [0.34230404, 0.35761793],
       [0.33852436, 0.64159963]])

In [390]:
len(foldsList)

4

In [403]:
foldsList = np.array_split(a, 4)

In [404]:
foldsList

[array(['a'], dtype='<U1'),
 array(['b'], dtype='<U1'),
 array(['c'], dtype='<U1'),
 array([], dtype='<U1')]

In [413]:
a = np.random.random((16,2))

In [414]:
a

array([[0.16957144, 0.74934758],
       [0.5807946 , 0.2636175 ],
       [0.65394189, 0.76193237],
       [0.69894282, 0.72840904],
       [0.1120781 , 0.10677641],
       [0.03919939, 0.83714064],
       [0.66882443, 0.94166158],
       [0.13469757, 0.40422441],
       [0.37952948, 0.24804356],
       [0.83577322, 0.22739505],
       [0.29891381, 0.96941245],
       [0.97218107, 0.97850869],
       [0.10194472, 0.24220638],
       [0.65841721, 0.08252444],
       [0.98004541, 0.72116683],
       [0.23907695, 0.10749088]])

In [415]:
foldsList = np.array_split(a, 4)

In [416]:
foldsList

[array([[0.16957144, 0.74934758],
        [0.5807946 , 0.2636175 ],
        [0.65394189, 0.76193237],
        [0.69894282, 0.72840904]]), array([[0.1120781 , 0.10677641],
        [0.03919939, 0.83714064],
        [0.66882443, 0.94166158],
        [0.13469757, 0.40422441]]), array([[0.37952948, 0.24804356],
        [0.83577322, 0.22739505],
        [0.29891381, 0.96941245],
        [0.97218107, 0.97850869]]), array([[0.10194472, 0.24220638],
        [0.65841721, 0.08252444],
        [0.98004541, 0.72116683],
        [0.23907695, 0.10749088]])]

In [417]:
for i in range(4):
    valData = foldsList[i]
    del foldsList[i]
    trData = np.vstack(foldsList)
    foldsList.insert(i, valData)

In [418]:
valData

array([[0.10194472, 0.24220638],
       [0.65841721, 0.08252444],
       [0.98004541, 0.72116683],
       [0.23907695, 0.10749088]])

In [420]:
len(foldsList)

4